<a href="https://colab.research.google.com/github/anaguilarar/gee_satellite_data/blob/master/Download%20Satellite%20Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip uninstall pyproj
!pip install pyproj
!pip install wget
!pip install geehydro
!pip install geopandas

Uninstalling pyproj-3.0.0.post1:
  Would remove:
    /usr/local/bin/pyproj
    /usr/local/lib/python3.6/dist-packages/pyproj-3.0.0.post1.dist-info/*
    /usr/local/lib/python3.6/dist-packages/pyproj.libs/libcurl-16e7dc3c.so.4.6.0
    /usr/local/lib/python3.6/dist-packages/pyproj.libs/libnghttp2-223fd912.so.14.17.1
    /usr/local/lib/python3.6/dist-packages/pyproj.libs/libproj-2bae3b87.so.19.2.0
    /usr/local/lib/python3.6/dist-packages/pyproj.libs/libsqlite3-69e6c45e.so.0.8.6
    /usr/local/lib/python3.6/dist-packages/pyproj.libs/libtiff-4e10f209.so.5.5.0
    /usr/local/lib/python3.6/dist-packages/pyproj.libs/libz-eb09ad1d.so.1.2.3
    /usr/local/lib/python3.6/dist-packages/pyproj/*
Proceed (y/n)? y
  Successfully uninstalled pyproj-3.0.0.post1
  Using cached https://files.pythonhosted.org/packages/e4/ab/280e80a67cfc109d15428c0ec56391fc03a65857b7727cf4e6e6f99a4204/pyproj-3.0.0.post1-cp36-cp36m-manylinux2010_x86_64.whl
     |████████████████████████████████| 972kB 6.7MB/s 
     |██████

In [2]:
!git clone https://github.com/anaguilarar/gee_satellite_data.git

Cloning into 'gee_satellite_data'...
remote: Enumerating objects: 234, done.
remote: Counting objects: 100% (234/234), done.
remote: Compressing objects: 100% (160/160), done.
remote: Total 234 (delta 126), reused 180 (delta 72), pack-reused 0
Receiving objects: 100% (234/234), 1.17 MiB | 9.89 MiB/s, done.
Resolving deltas: 100% (126/126), done.


In [3]:
import os
os.chdir("gee_satellite_data")

In [4]:
import ee
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=3WjvxHbU5jmAFaGcALYj8IB1ujT7lHqjfJjzB89_0Kc&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g6mkU7vzP9O-2amSWyLsR8FRTcwNZcNNB4xOP_OqctiMIdccZ8SmW4

Successfully saved authorization token.


In [11]:
import pandas as pd
from scripts import gee_satellite_data
from scripts import gee_functions
from scripts import gis_functions
from scripts import general_functions

## Download Sentinel 1 data

In [12]:
querydata = gee_satellite_data.get_gee_data("2016-02-01",
                                            "2016-09-30", 
                                            "data/col_t3.shp", 
                                            "sentinel1", bands = ['VV'])

In [13]:
querydata.summary

,dates,cover_percentage
0,2016-03-26 23:13:23,99.558320
1,2016-05-13 23:13:30,99.559609
2,2016-09-10 23:13:37,99.556874
3,2016-02-07 23:13:28,99.559613
4,2016-02-14 10:42:59,99.558320
5,2016-03-02 23:13:28,99.558167
6,2016-03-09 10:42:58,99.556873
7,2016-04-02 10:43:00,99.558320
8,2016-04-19 23:13:29,99.561045
9,2016-04-26 10:43:00,99.558310


Due to Sentinel 1 mission has two satellites (a and b), it can orbit the earth in a different orientation (Ascending and Descending), the  6 days data can be summarised averaging over a time window. 

In [15]:
querydata.reduce_collection_by_days(10)
querydata._dates_reduced

0    2016-03-30 04:58:12
1    2016-04-19 23:13:29
2    2016-04-26 10:43:00
3    2016-05-13 23:13:30
4    2016-05-20 10:43:02
5    2016-06-10 04:58:18
6    2016-06-30 23:13:33
7    2016-07-07 10:43:05
8    2016-07-28 04:58:20
9    2016-08-17 23:13:36
10   2016-08-24 10:43:07
11   2016-09-10 23:13:37
dtype: datetime64[ns]

In [16]:
imagetoplot = querydata.image_collection.toList(querydata.image_collection.size()).get(2)

radarParams = {'gamma': 1.3, 
                   'min': -25,
                   'max': 5,
                   'bands': ['VV']
                   }

gee_satellite_data.plot_eeimage(imagetoplot, radarParams, querydata.geometry, zoom = 11)


if the time window reduction step was completed, the code will download all the images that were calculated in that step. Otherwise, it will download the images that were acquired from the inital query

## Download Landsat 8 Surface Reflectance data

In [19]:
landsat8 = gee_satellite_data.get_gee_data("2018-01-01",
                                           "2018-12-31", 
                                           "data/col_t3.shp", 
                                           "landsat8_t1sr", 
                                            bands = ['B2', 'B3', 'B4', 'B5', 'B10', 'B11'],
                                            cloud_percentage= 70)

In [20]:
landsat8.summary

,dates,cover_percentage
0,2018-02-13 00:00:00,2.539184
1,2018-03-01 15:12:27,46.204227
2,2018-03-17 00:00:00,92.975099
3,2018-08-08 15:12:07,99.550840
4,2018-08-24 00:00:00,62.893990
5,2018-09-09 00:00:00,97.080620
6,2018-11-28 15:13:04,0.000000
7,2018-12-30 00:00:00,88.612941


In [21]:

truecolorParams = {'gamma': 1.3, 
                   'min': 57,
                   'max': 2000,
                   'bands': ['B4','B3','B2']
                   }

imagetoplot = landsat8.image_collection.toList(landsat8.image_collection.size()).get(5)

gee_satellite_data.plot_eeimage(imagetoplot, truecolorParams, landsat8.geometry, zoom = 11)



### Adding vegetation index layers

In [22]:
landsat8.add_vi_layer("ndvi")

In [23]:
ndviParams = {     'min': 0,
                   'max': 1,
                   'palette': ['#FF0000', '#00FF00'],
                   'bands': ['ndvi']
                   }

imagetoplot = landsat8.image_collection.toList(landsat8.image_collection.size()).get(5)

gee_satellite_data.plot_eeimage(imagetoplot, ndviParams, landsat8.geometry, zoom = 11)

### Registrate Landsat 8 images using a sentinel surface reflectance image

In [24]:
displacement, s2image, l8image  = landsat8.l8_displacement()


the S2 image reference was found in 2019-06-07


In [ ]:
landsat8.displace_landsatcollection(displacement)

the image collection was resgistered


In [25]:
import folium
import geehydro

registered = l8image.displace(displacement);

truecolorParams = {'gamma': 1.3, 
                   'min': 57,
                   'max': 2000,
                   'bands': ['B4','B3','B2']
                   }

centergeometry = gis_functions.geometry_center(landsat8.geometry)


Map = folium.Map(location=[centergeometry[1],
                           centergeometry[0]], zoom_start=11)

eegeom = gis_functions.polygon_fromgeometry(landsat8.geometry)
eegeom = gee_functions.geometry_as_ee(eegeom)

Map.addLayer(ee.Image(s2image), truecolorParams, 'gee image s2')
Map.addLayer(ee.Image(l8image), truecolorParams, 'gee image ls')
Map.addLayer(ee.Image(registered), truecolorParams, 'gee image ls registered')


Map.addLayer(ee.Image().paint(eegeom, 1, 3), {}, 'region of interest:')


Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)

Map

### Download data

In [ ]:
gee_satellite_data.download_gee_tolocal(landsat8, ## gee_data class
                                        'gee_satellitedata/l8_processed', ## output directory
                                        "col_t3", ## suffix identifiacation
                                        10, # spatial resolutions
                                        bands = ['B2', 'B3', 'B4', 'B5', 'B10', 'B11', 'ndvi'], ## bands to be exported
                                        cover_percentage= 75 ## minimum cover percentage
                                       )

-1 / unknownthe gee_satellitedata/l8_processed\l8_t1sr_20160327_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/l8_processed\l8_t1sr_20160530_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/l8_processed\l8_t1sr_20160717_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/l8_processed\l8_t1sr_20160903_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/l8_processed\l8_t1sr_20161021_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/l8_processed\l8_t1sr_20161106_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/l8_processed\l8_t1sr_20161122_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/l8_processed\l8_t1sr_20170314_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/l8_processed\l8_t1sr_20170602_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/l8_processed\l8_t1sr_20170704_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/l8_processed\l8_

## Download Sentinel 2 Surface Reflectance data

In [26]:
sentinel2 = gee_satellite_data.get_gee_data("2019-06-01", ## start date
                                            "2019-09-28", ## end query date
                                            "data/col_t3.shp", ## region of interest
                                            "sentinel2_sr", ## mission
                                            cloud_percentage= 80 ## cloud percetange per image 
                                           )

In [27]:
sentinel2.summary

,dates,cover_percentage
0,2019-06-08 00:00:00,98.326225
1,2019-06-13 15:32:02,19.156624
2,2019-07-03 00:00:00,79.586576
3,2019-07-13 00:00:00,53.914557
4,2019-07-23 00:00:00,0.000000
5,2019-08-02 00:00:00,79.009143
6,2019-08-07 00:00:00,72.782221
7,2019-08-12 00:00:00,99.544819
8,2019-08-17 00:00:00,52.994298
9,2019-08-22 00:00:00,99.107698


In [28]:
truecolorParams = {'gamma': 1.3, 
                   'min': 57,
                   'max': 2000,
                   'bands': ['B4','B3','B2']
                   }

imagetoplot = sentinel2.image_collection.toList(sentinel2.image_collection.size()).get(9)

gee_satellite_data.plot_eeimage(imagetoplot, truecolorParams, sentinel2.geometry, zoom = 11)

### Adding vegetation index layers

In [29]:
sentinel2.add_vi_layer("ndvi")

In [30]:
ndviParams = {     'min': 0,
                   'max': 1,
                   'palette': ['#FF0000', '#00FF00'],
                   'bands': ['ndvi']
                   }

imagetoplot = sentinel2.image_collection.toList(sentinel2.image_collection.size()).get(9)

gee_satellite_data.plot_eeimage(imagetoplot, ndviParams, sentinel2.geometry, zoom = 11)

### Download data

In [ ]:
gee_satellite_data.download_gee_tolocal(sentinel2, ## 
                                        'gee_satellitedata/s2_processed', ## outputpath 
                                        "col_t3", ## a reference for the area that was query
                                        10, ## the spatial resolution in meters
                                        bands = ['B2', 'B3', 'B4', 'B8', 'ndvi']
                                       )

-1 / unknownthe gee_satellitedata/s2_processed\s2_l2a_20190608_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/s2_processed\s2_l2a_20190613_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/s2_processed\s2_l2a_20190703_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/s2_processed\s2_l2a_20190713_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/s2_processed\s2_l2a_20190723_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/s2_processed\s2_l2a_20190802_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/s2_processed\s2_l2a_20190807_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/s2_processed\s2_l2a_20190812_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/s2_processed\s2_l2a_20190817_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/s2_processed\s2_l2a_20190822_col_t3_10m file was downloaded
-1 / unknownthe gee_satellitedata/s2_processed\s2_l2a_201908